In [91]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import codecs

In [46]:
home = 'http://www.chinavitae.com'
institution = 'http://www.chinavitae.com/institution/'
start_html = requests.get(institution)

In [47]:
soup = BeautifulSoup(start_html.text, 'lxml')

In [51]:
""" 
    Find <a> and 'href' attributes 
    Output: a_tag_dic = {link:Category_name}
"""
def find_a_tag(soup):
    a_tag = soup.find_all('a')
    a_tag_dic = {}
    for a in a_tag:
        if a.has_attr('href'):
            attr = a['href']
            attr_search = re.compile('/institution/.+')
            attr_str = re.match(attr_search, attr)
            if attr_str:
                a_tag_dic[home+attr_str.group()] = a.text
    return a_tag_dic

In [54]:
a_tag_dic = find_a_tag(soup)

In [97]:
"""
    First layer: 'http://www.chinavitae.com/institution' and their direct children
"""
list1 = []
for url, val in a_tag_dic.iteritems():
    url_html = requests.get(url)
    url_soup = BeautifulSoup(url_html.text, 'lxml')
    list1.append(find_a_tag(url_soup))

In [96]:
print list1[0]

{'http://www.chinavitae.com/institution/pc': u"People's Congress at National, Provincial, and Local Levels", 'http://www.chinavitae.com/institution/pc/1610.': u"People's Congress (NPC)"}


In [71]:
"""
    Fetch all url and name to list of list
"""
list3 = []
for dic in list1:
    list2 = []
    for url, val in dic.iteritems():
        url_html = requests.get(url)
        url_soup = BeautifulSoup(url_html.text, 'lxml')
        list2.append(find_a_tag(url_soup))
    list3.append(list2)

In [81]:
"""
    Get category and id from url
"""
def get_category_and_id_from_url(url):
    url_cat = ''
    url_id = ''
    base = 'http://www.chinavitae.com/institution/'
    piece = url.split('/')

    if piece[-1][0].isdigit():
        url_id = piece[-1]
        url_cat = piece[-2]
    else:
        url_id = url_cat = piece[-1]
    return [url_cat, url_id]

In [94]:
"""
    Write id into files
"""
everything = list3
myfile = codecs.open('id-list.txt', 'w', 'utf-8')
i = 0
for category in list3:
    for level in category:
        for url, name in level.iteritems():
            [tmp_cat, tmp_id] = get_category_and_id_from_url(url)
            myfile.write(tmp_cat + ', ' + tmp_id + ', ' + name + '\n')
            i += 1
myfile.close()
print i

9765
